In [ ]:
# IMPORTANDO AS BIBLIOTECAS

import pandas as pd
import psycopg2
import email
import openpyxl
import os
from sqlalchemy import create_engine        

pd.set_option('display.max_columns',100)

In [ ]:
caminho_drive_GARE = r"G:\.shortcut-targets-by-id\1YZLLTFOuLniE6hO7WNHXl2c3MSa4TOv6\X. GAME11\GUARDIAN Institucional\5. Planejamento & Processos\2. 2023\4. Research\2025\5. Inteligência Passivos Guardian\1. ARQUIVOS B3\BRUTOS\GARE\11. NOV"
caminho_drive_GAME = r"G:\.shortcut-targets-by-id\1YZLLTFOuLniE6hO7WNHXl2c3MSa4TOv6\X. GAME11\GUARDIAN Institucional\5. Planejamento & Processos\2. 2023\4. Research\2025\5. Inteligência Passivos Guardian\1. ARQUIVOS B3\BRUTOS\GAME\11. NOV"
df_GARE11 = pd.read_excel(f"{caminho_drive_GARE}\\GARE 14.11.xlsx")
df_GAME11 = pd.read_excel(f"{caminho_drive_GAME}\\GAME 14.11.xlsx")

In [ ]:
# VARIAVEL PARA RENOMEAR AS COLUNAS
cols_rename = {'CPF/CNPJ':'DOCUMENTO', 'Quantidade':'TOTAL', 'Tipo':'Classe_BVMF', 'Nome':'NOME'}

#VARIAVEL PARA EXCLUIR AS COLUNAS
cols_drop = ['ISIN', 'Fundo', 'Data', 'Data Posição']


# FUNÇÃO PARA CATEGORIZAR PF E PJ
def identificadorid(x):
    if x == 'F00101' or x == 'F01001':
        return 'PF'
    else:
        return 'PJ'

# FUNÇÃO PARA CRIAR A COLUNA DE Classe_BVMF
def categoria(ct):
    if ct == 'F00101':
        return 'PESSOA FISICA'
    elif ct == 'J00202':
        return 'SOCIEDADE LIMITADA, ECONOMIA MISTA, ANÔNIMA, POR COTAS, ETC'
    elif ct == 'J00203':
        return 'FUNDO DE INVESTIMENTO IMOBILIÁRIO'
    elif ct == 'J00501':
        return 'FUNDO DE INVESTIMENTO (EXCETO FUNDO DE INVESTIMENTO IMOBILIÁRIO)'
    elif ct == 'J01002':
        return 'INVESTIDOR NÃO-RESIDENTE RESOLUÇÃO N 2.689 - PJ'
    elif ct == 'J00901':
        return 'TEMPLO DE QUALQUER CULTO'
    elif ct == 'F01001':
        return 'INVESTIDOR NÃO-RESIDENTE RESOLUÇÃO N 2.689 - PF'
    elif ct == 'J00405':
        return 'SOCIEDADE CORRETORA DE TÍTULOS E VALORES MOBILIÁRIOS E CÂMBIO'
    elif ct == 'J00201':
        return 'SOCIEDADE LIMITADA, ECONOMIA MISTA, ANÔNIMA, POR COTAS, ETC'
    elif ct == 'J00402':
        return 'BANCO DE INVESTIMENTOS'
    elif ct == 'J00601':
        return 'FUNDO DE PREVIDÊNCIA PRIVADA - ABERTO'
    elif ct == 'J00904':
        return 'INSTITUIÇÃO DE EDUCAÇÃO E ASSITÊNCIA SOCIAL, SEM FINS LUCRATIVOS'
    else:
        return 'ERRO DE CLASSIFICAÇÃO'

#FUNÇÃO PARA TRATAR OS CPFS COM NUMERO FALTANDO NA COLUNA DE ID_INVESTIDOR
def tratar_doc(TP, doc):
    
    doc = str(doc)
    doc = doc.split('.')[0]

    if TP == 'PF':
        doc = doc.zfill(11)
        return doc
    else:
        doc = doc.zfill(14)[:14]
        return doc

#def excluir_5digitos(cpf_cnpj):
    return str(cpf_cnpj)[5:]    


# FUNÇÃO PARA TRATAR A TABELA DE MANEIRA GERAL
def tratar_colunas(x):
    #RENOMEIA
    x = x.rename(columns = cols_rename)
    #CRIA A COLUNA DE PF E PJ
    x['PF_PJ'] = x.Classe_BVMF.apply(identificadorid)
    #MAPEANDO A COLUNA DE TIPO
    x['Classe_BVMF'] = x.Classe_BVMF.apply(categoria)
    #MOVENDO AS COLUNAS PARA FORMATAR A TABELA DE MANEIRA CORRETA
    x = x[['DOCUMENTO', 'NOME', 'PF_PJ', 'Classe_BVMF', 'TOTAL']]
    #RETIRA OS ESPAÇOS DO NOME
    x['NOME'] = x['NOME'].str.strip()
    #APLICANDO A FUNÇÃO PARA FORMATAR O CPF E CNPJ NA COLUNA ID_Investidor
    x['DOCUMENTO'] = x['DOCUMENTO'].astype(str)
    #APLICANDO A FUNÇÃO PARA FORMATAR O CPF E CNPJ NA COLUNA ID_Investidor 
    x['DOCUMENTO'] = x.apply(lambda row: tratar_doc(row['PF_PJ'], row['DOCUMENTO']), axis = 1)
    #EXCLUINDO OS 5 PRIMEIROS DIGITOS
    #x['DOCUMENTO'] = x.DOCUMENTO.apply(excluir_5digitos)
    #RETIRA OS ESPAÇOS DO ID_Investidor
    x['DOCUMENTO'] = x['DOCUMENTO'].str.strip()
    #CRIANDO A CHAVE DE LIGAÇÃO
    x['CHAVE'] = x['DOCUMENTO'] + x['NOME'].str[:30]
    #ORDENANDO AS COLUNAS
    x = x[['CHAVE', 'DOCUMENTO', 'NOME', 'PF_PJ', 'TOTAL']]
    
    

    return x

In [ ]:
# TRATANDO MEU DATAFRAME
df_GARE = tratar_colunas(df_GARE11)
df_GAME = tratar_colunas(df_GAME11)

In [ ]:
df_agrupado_GARE = df_GARE.groupby(['CHAVE']).size().reset_index(name='Quantidade')

# Exibir as linhas onde há mais de uma ocorrência (ou seja, duplicados)
df_duplicados_GARE = df_agrupado_GARE[df_agrupado_GARE['Quantidade'] > 1]

df_agrupado_GAME = df_GAME.groupby(['CHAVE']).size().reset_index(name='Quantidade')

# Exibir as linhas onde há mais de uma ocorrência (ou seja, duplicados)
df_duplicados_GAME = df_agrupado_GAME[df_agrupado_GAME['Quantidade'] > 1]

# Exibir o resultado
print(df_duplicados_GARE)
print(df_duplicados_GARE)

In [ ]:
df_GARE = df_GARE.groupby(['DOCUMENTO'], as_index=False).agg({'CHAVE':'first',
                                                       'DOCUMENTO':'first',
                                                       'NOME':'first',
                                                       'PF_PJ':'first',
                                                       'TOTAL':'sum'})

df_GAME = df_GAME.groupby(['DOCUMENTO'], as_index=False).agg({'CHAVE':'first',
                                                       'DOCUMENTO':'first',
                                                       'NOME':'first',
                                                       'PF_PJ':'first',
                                                       'TOTAL':'sum'})

In [ ]:
#SALVANDO O DATAFRAME EM .XLSX EM UMA PASTA ESPECIFICA

#MAPEANDO O CAMINHO E SALVANDO A PLANILHA TRATADA COMO .XLSX
pasta_destino_GARE = r'G:\.shortcut-targets-by-id\1YZLLTFOuLniE6hO7WNHXl2c3MSa4TOv6\X. GAME11\GUARDIAN Institucional\5. Planejamento & Processos\2. 2023\4. Research\2025\5. Inteligência Passivos Guardian\1. ARQUIVOS B3\FORMATADOS\GARE\10. OUT'
pasta_destino_GAME = r'G:\.shortcut-targets-by-id\1YZLLTFOuLniE6hO7WNHXl2c3MSa4TOv6\X. GAME11\GUARDIAN Institucional\5. Planejamento & Processos\2. 2023\4. Research\2025\5. Inteligência Passivos Guardian\1. ARQUIVOS B3\FORMATADOS\GAME\10. OUT'
#CASO A PASTA NÃO EXISTA UMA SERA CRIADA IMEDIANTAMENT
os.makedirs(pasta_destino_GARE, exist_ok=True)
#INSERE EM UMA VARIAVEL O CAMINHO E O NOME DO ARQUIVO
caminho_arquivo_excel = os.path.join(pasta_destino_GARE, '09.10_teste.xlsx')
#EXPORTA O ARQUIVO PARA .XLSX E O SALVA NA PASTA QUE O CAMINHO INDICA
df_GARE.to_excel(caminho_arquivo_excel, index=False)

#CASO A PASTA NÃO EXISTA UMA SERA CRIADA IMEDIANTAMENT
os.makedirs(pasta_destino_GAME, exist_ok=True)
#INSERE EM UMA VARIAVEL O CAMINHO E O NOME DO ARQUIVO
caminho_arquivo_excel = os.path.join(pasta_destino_GAME, '09.10_teste.xlsx')
#EXPORTA O ARQUIVO PARA .XLSX E O SALVA NA PASTA QUE O CAMINHO INDICA
df_GAME.to_excel(caminho_arquivo_excel, index=False)

print('Arquivos salvo com sucesso')